# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [90]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [91]:
city_df = pd.read_csv("../output_data/cities.csv")
city_df.dropna()




,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [92]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [93]:
# Store 'Lat' and 'Lng' into  locations
locations = file[["Lat", "Lng"]]
humidity = file["Humidity"].astype(float)

In [94]:
#Plot heatmap
fig = gmaps.figure(center=(45.0, -5.0), zoom_level=2)
#Add heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=np.max(humidity), point_radius=3)

fig.add_layer(heat_layer)
print(fig)

Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [95]:
city_df = city_df.loc[(city_df["Wind Speed"] <= 10) & (city_df["Cloudiness"]== 0) & \
                  (city_df["Max Temp"] >= 70) & (city_df["Max Temp"] <= 85)].dropna()
city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
316,316,gunjur,0,GM,1558378848,61,13.18,-16.76,80.60,9.17
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [96]:
#Store into variable named hotel_df
hotel_df = city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
125,arraial do cabo,BR,-22.97,-42.02,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
316,gunjur,GM,13.18,-16.76,
444,birjand,IR,32.86,59.22,
518,bonito,BR,-21.13,-56.49,


In [97]:
# dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


{
    "html_attributions": [],
    "next_page_token": "ATtYBwIdtmojL0bT4PRwsBXOh_NtHzRexIKSOZX--b-1Diwhql2iWm7PrOpytkAY4oK1H1PDAm78pAXGyUd-gRRxyc2JCGHeKHSELMs9tQbnqbHyYZSkFwmmwKVTE53hmxGVgz8bwj4H7mTOoVDFmrI-q98EC5pYBwd6bqD_PFWrPBtt--zXUAzYbZ7WTQ44uei4BStDcqjL-5U2XiY1I4OSBhngNEBz9tT_EFtYFfdkSi0jTWghwGRFjHAEzNGuZiFt7BAJNtFx3pMVYd5ge44fDSefMpV5ERYvDQG7VA6egeysdSMpuy2wao6k2wVnGtbcc0xlqna3kRM5xR5aatRq5QPmXHk1GPF9V0iLKALYvmMoArTSOix4I-81RFC8uSa87lqqWFqo8PW9tvAbbj1v8MBKzsBEt_viOmJ3lau57P_7ad1VZiu_rnYjgcF5Y-WW",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.6441728,
                    "lng": -46.1058822
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.64274842010728,
                        "lng": -46.10453982010728
                    },
                    "southwest": {
                        "lat": -20.6454

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKoR3ktfu_SOedOG-OE662RvgHS94Xot7QEOBm5HR1UwK56VZi2SZqyNPA-Ro8OG2k-obyAuHDS9FB3OoFmdcHKXjByjdk5RkZ-WsY-g26rI1rWwSR492Vk3XQxMJdm5s-AAhl2np3gvSIkQqg20ov8hRA5AIGFt-H0IpzIqPutxtu9CaNShzRtM_xiLNy-kodOeT3ucnyG5jB2nLzj4aE9ywlX--njb-80XBppbNtF929cprZlRyzwj_AhIwD5xqAPnM98M97hmGYeT81GuR8WI5u44jlvx9LjX8a8d6kSpHnRmj6Wd4Ph24fhv3FUf3_PZK0OM2rT39kWcgrMyQmIeqT_qrCY8Y83IgWZpmXkEAxUkfpx1wL6EuZfgwkgfclw6DVFgP0_tqoPp7Rw0TrrrgRs6a8WsmHVsYxgrBsgi31WBu6Tp72LnznhkQK4",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -29.7413463,
                    "lng": -50.0030861
                },
                "viewport": {
                    "northeast": {
                        "lat": -29.73992642010727,
                        "lng": -50.00188702010728
                    },
                    "southwest": {
                        "lat": -29.7426

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJZ4aA57tcjjrneckqHoIwLmjQmhobyUdSvP7xE8s5yddovRwwnDE7nGRZWX5BimfuuXFcpamcJVS6jVzRJNUFjYjCK7Io1hecOSgo4CBMA7idOiTBTE1gl8aAlQvfqsKvb_hbZYnN2r8qE3CCBwdLRpNSGx_sdIhuAnUaywM_XQ4o4DVtSSTvXpmzeC2kVnelnnVrJai2JnthrIP55lm3PkA0lULYgseGRS7obuFcxD0SUV2GFmpq7mG9kxOxQr-wuXh8tqyvTmahJMg78xxE2KClqIuO8Dmk0jApstJLQX7Ng-AfhgZyXv0hV_wrPw0cn6qz9xFpm2CUcWw_k9bgjTvLkqUQ6btGKjyi5ur2HiBRMoClp51E3QANGYTGZaE7UWHT-UumucCvCc1Q-VD9eBWLWAUrwZbGBFiZ49PP2BrKqSU9jdc2Uwt_EWbd9",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.8596134,
                    "lng": -41.98712159999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.85827152010728,
                        "lng": -41.98584777010728
                    },
                    "southwest": {
                        "lat": -

{
    "html_attributions": [],
    "next_page_token": "ATtYBwItrSHhXqMVbTz8NmJ88CAmfKQmodN3anvcgNmfnGUvQS4W2g7RM7bQG8IU2jPPX-KawNxsqvBazWaEBKrnBN4P1XkflzrRL77rP-gv_BizD_0Pxzto7xMB8j1WyvdMfTVlTL6evIfdygxLRqA_Onp7XbaHniptlSCMlM1F8xBlMfVRRDnMOqzIrnWOiiVkWoLEdwm5PEChWX2_cyVkEv-9ZW_aou5Guqy7L3folUtCUb65bngwKQ5e1SLwJVrbm_hrSHjdnn57OKn-uA1kvhKGzmssETnKU5ow90Ic0xUPQvFzz0eZtVXb-gE5b1sbkm12VWalVT1y05BtxIRe1PZwNbwTGSPSWOEPBhQcD7dK_6yOSUv7Q6iQfYKJAanlGeH0BUmWEwJTt1Trn7-z7LYx6GearOSRhFZ1NOdhKXVwQ1X-AoH04B-fDExiNtMm",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.3000259,
                    "lng": 44.2679367
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.29871172010727,
                        "lng": 44.26928077989272
                    },
                    "southwest": {
                        "lat": -20.301411

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 14.991011,
                    "lng": 74.04016899999999
                },
                "viewport": {
                    "northeast": {
                        "lat": 14.99234872989272,
                        "lng": 74.04161817989272
                    },
                    "southwest": {
                        "lat": 14.98964907010728,
                        "lng": 74.03891852010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "The LaLiT Golf & Spa Resort Goa",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 1067,
                    "html_attributions": [
         

...

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJ_MMtbiKSDHith9kHzZPkZjl_RX4HI8W6RfDGjBUWOCqMbRk3JImhjTTDixX6e2RgTy4urhKXFs3GNjeQRe4k3zpAQVZq_1-lQ2vrMfyOKtiEcMcfKUNLvGmapATpi4O_Wnl9TXFTlr65jtG-hCMIYA6hSvDzn1Vhl9ur3K-UzxQ2Gzmt8D6zwize5hhopYW8_xqoQnqce_yFLrxyIYI4OplWnHXpznzei_75fcCdqtMpWOWm-6qeXLS9dK1fX0sgvuCO2NRotH8GkqwRk0-3qaCNJdza2an_TTG9HPJ0TTOebd7nyteyFqn8hW34C9IiY_4koA4KmtiuvBKCTE5NZ23Vhb-A3dIaipsbHr38zG3wHjn4FyLqkNSV_5s_7MtwrZYAyOEXh5J1LY9vB1DEFCqEGm91uS8rkbPxBaP46cS12WQbrrkf7grvcKPT_",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 13.4252778,
                    "lng": -16.7327778
                },
                "viewport": {
                    "northeast": {
                        "lat": 13.42670027989272,
                        "lng": -16.73138087010728
                    },
                    "southwest": {
                        "lat": 13.4240006

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 32.8275139,
                    "lng": 59.183111
                },
                "viewport": {
                    "northeast": {
                        "lat": 32.82883362989273,
                        "lng": 59.18451257989271
                    },
                    "southwest": {
                        "lat": 32.82613397010729,
                        "lng": 59.18181292010727
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Birjand Mountain Grand",
            "photos": [
                {
                    "height": 4032,
                    "html_attributions": [
                        "<a href=\"https://maps.google.com/maps/contrib/111530001407497358408\">Javad Re

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLIe5_-2un1Yxjk7cOZXiN5yKMzgRP6obBS82qQJcWpyMVIEmzCJF_E3cm5TtBdokANt9pnXzkCCTXLMN9VYeWoEQ8WzP6iORdqoBDVZjABlNd2xYjdSNQikve6GWYRWAbh6PSq9cRG0u7o8x_DKigYT8-hXgFKHfCS5FSQfbVMrTzyM3rFL4O5iBj3H8MCSkSvIJ74vYMBrOqo03TawKTxZUaXg6kY4LcfzDIE2cthgdpH1Du_UFowNSXP_r6FZBwbckPW3ecfsuGX0R1NXwXGwyRMKOc0mL6UYizidERB13HfLXkzpi_XTJFKuRMnFrp72_1EAR2faiXaUfX3dnnw681WF3Si59a-iQUiCi2bDqUfWdoRqRlf2ZHA_V8rJzVWmqwD9PJVcgxWaz0EteY5hFX4jr0aGWyOdrhnEsTymNNWSoRY2kw32OmMmEac",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.109918,
                    "lng": -56.517364
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.10842367010728,
                        "lng": -56.51634372010728
                    },
                    "southwest": {
                        "lat": -21.111123

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIkcem67xxhRPSQ6VhKc_Gyz9zTUbzbjajULeH-WGoMvsWNFG4eZ0p2tC1o7kmsesSiO47gHs2IrOuugKHIUZgJnpvEHphzrVIVwrMz5-97oT_wKTdp1hlg1fwOHOvA699Q5ze0TbCb_kpPV_oo5KIdP2tgTb9_X9aDOuJ9zdCUO-wPgBN5QfQ9AK3eG8N3aODTA5RFF2MB7v417YzATvqrMZhImLj-fGLBhZY9F97xgypoogjsY_T1j805ww8GSaFs1MzUS-JuhO4_XtenG1iSfWCr9VOefPqQYvGwGxU5fsVr86VohQRA23VE5pXp6yDMQVO5z77b9YXI6st_UlnYy328h9dXdSWZvs9LGP03py77Y_TxmklJqwRIoWJEYhMW1bUF3fpkuggZHxx4chZwXiEVhl-Ieyd2VRgG2OjALI9-CQ4lz2gtHjOrX-io",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 20.2499439,
                    "lng": -105.5830013
                },
                "viewport": {
                    "northeast": {
                        "lat": 20.25119597989272,
                        "lng": -105.5816417201073
                    },
                    "southwest": {
                        "lat": 20.248496

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 33.9980282,
                    "lng": 36.218071
                },
                "viewport": {
                    "northeast": {
                        "lat": 33.99938097989271,
                        "lng": 36.21940947989273
                    },
                    "southwest": {
                        "lat": 33.99668132010726,
                        "lng": 36.21670982010728
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png",
            "name": "Kanaan Group Hotel",
            "opening_hours": {
                "open_now": true
            },
            "photos": [
                {
                    "height": 2988,
                    "html_attributions": [
                        "<a h

In [98]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,Balneário do Lago Hotel
88,cidreira,BR,-30.17,-50.22,Hotel Araçá
125,arraial do cabo,BR,-22.97,-42.02,Hotel La Plage
138,morondava,MG,-20.30,44.28,Palissandre Cote Ouest resort & SPA
139,vaini,IN,15.34,74.49,The LaLiT Golf & Spa Resort Goa
176,nador,MA,35.17,-2.93,"Hotel Marchica Lagoon Resort, Nador Morocco"
297,mogok,MM,22.92,96.51,Mount Mogok Hotel
316,gunjur,GM,13.18,-16.76,Coco Ocean Resort & Spa
444,birjand,IR,32.86,59.22,Birjand Mountain Grand
518,bonito,BR,-21.13,-56.49,Zagaia Eco Resort


In [105]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [106]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)


# Display figure
print(fig)

Figure(layout=FigureLayout(height='420px'))
